In [1]:
%run ../talktools.py

## GPs for Measuring the Rotation Periods of Stars
### A summary of Angus et al. 2018
### AY 128/256 (UC Berkeley, 2020)


## Motivation

* The disc-integrated flux of a spotted, rotating star often varies in a non-sinusoidal and quasi-periodic (QP) manner, due to active re- gions on its surface that rotate in and out of view. Complicated surface spot patterns produce non-sinusoidal variations, and the finite lifetimes of these active regions and differential rotation on the stellar surface produce quasi-periodicity.

* Strictly periodic sinusoid is therefore not necessarily a good model choice.

<img src="img/angus_f1.png" ></img>

Fig 1. Light curve of KIC 5809890, an active star with a rotation period of ∼ 30.5 d. The solid line shows a ﬁt to the data using a GP model with a QP covariance kernel function.

## Possible Methods

* Standard ways to model this type of data to meaure a period:
 - Lomb-Scargle (LS)
 - Auto Correlation Function (ACF)
 
* Why or why are they a good or bad choice?

## Gaussian Processes

* Gaussian processes: very flexible, can be expensive to compute, can deal with multiple (quasi-)periodic processes in the data

* In this work, we test an effective model for rotationally modulated stellar light curves that captures the salient behaviour but is not physically motivated – although some parameters may indeed be interpreted as physical ones.

* An ideal effective model should have a small number of non-degenerate parameters and be ﬂexible enough to perfectly capture non-sinusoidal and QP behaviour.

* They are useful in regression problems involving any stochastic process, speciﬁcally when the probability distribution for the process is a multivariate Gaussian. If the probability of obtaining a data set is a Gaussian in N dimensions, where N is the number of data points, a GP can describe that data set.

* An in-depth introduction to GPs is provided in Rasmussen & Williams (2005).

* GP models parametrize the covariance between data points by means of a kernel function.


<img src="img/angus_f1.png" ></img>

* This is a relatively active star that rotates once every ∼30.5 days, with stochastic variability typical of Kepler FGK stars. 

* Clearly, data points in this light curve are correlated. Points close together in time are tightly correlated, and points more widely separated are loosely correlated. 

* A GP models this variation in correlation as a function of the separation between data points; that is, it models the covariance structure rather than the data directly. 

* This lends GPs their ﬂexibility – they can model any time series with a similar covariance structure. 

* In addition, a very simple function can usually capture the covariance structure of a light curve, whereas modelling the time series itself might require much more complexity. 

* A range of covariance models, or kernel functions, can describe stellar variability. For example, the most commonly used kernel function, the ‘squared exponential’ (SE), deﬁned as follows, could adequately ﬁt the KIC 5809890 light curve:

$k_{i,j} = A \exp \left(-\frac{(x_i - x_j)^2}{2l^2} \right)$

* $A > 0$ is the amplitude of covariance, 

* $l$ is the lengthscale of exponential decay 

* $x_i − x_j$ is the separation between data points.

* The SE kernel function has the advantage of being very simple, with just two parameters, $A$ and $l$.

* If $l$ is large, two data points far apart in $x$ will be tightly correlated, and if small they will be loosely correlated.

* The SE kernel function produces functions that are inﬁnitely differentiable, making it possible to model a data set and its derivatives simultaneously. __Why might this matter?__

* __Problem with the SE kernel__: It does not well describe the covariance in stellar light curves, nor is it useful for the problem of rotation period inference because it does not capture periodic behaviour.

### The Quasi Periodic Kernel


$ k_{i,j} = A \exp \left[-\frac{(x_i - x_j)^2}{2l^2} -
    \Gamma^2 \sin^2\left(\frac{\pi(x_i - x_j)}{P}\right) \right] + \sigma^2
    \delta_{ij}. $
  
* Product of two kernels: SE and sinusodal  
  
* SE kernel function, which describes the overall covariance decay

* Exponentiated, squared, sinusoidal kernel function that describes the periodic covariance structure

* P can be interpreted as the rotation period of the star

* $\Gamma$ controls the amplitude of the $sin^2$ term.

* Large values of $\Gamma$ lead to periodic variations with increasingly complex harmonic content

 - If $\Gamma$ is very large, only points almost exactly one period away are tightly correlated and points that are slightly more or less than one period away are very loosely correlated.

 - If $\Gamma$  is small, points separated by one period are tightly correlated, and points separated by slightly more or less are still highly correlated.

* The parameter $\sigma$ captures white noise by adding a term to the diagonal of the covariance matrix. 
 - This can be interpreted to represent underestimation of observational uncertainties
 - If the uncertainties reported on the data are too small, it will be non-zero
 - It can capture any remaining ‘jitter’, or residuals not captured by the effective GP model.
 
* There are many ways to construct a QP kernel function, involving a range of choices for both the periodic and aperiodic components of the model.
 - The kernel function presented above reproduces the behaviour of stellar light curves
 - Other model choices can do so as well.
 - Angus et al. do not attempt to test any other models in this paper, noting only that this kernel function provides an adequate ﬁt to the data.

## Fitting the Data Philosophy

* Performing inference with GPs is computationally expensive.
 - Is it worth spending the extra computation power to get slightly more accurate, probabilistic rotation periods for a large number of stars?
 - The likelihood of the model could be maximized to ﬁnd the maximum-likelihood value for $P$.
 - We explore the full posterior PDFs using a Markov Chain Monte Carlo (MCMC) procedure.

* Such posterior exploration importantly provides a justiﬁed uncertainty estimate.
 - Probabilistic inference allows one to marginalize over nuisance parameters while formally taking any uncertainty into account when measuring the quantity of interest.
 - Especially important as the signal-to-noise ratio decreases and the rotation period is no longer tightly constrained.  
 - Also, as sampling becomes uneven.
 - Probabilistic measurements with full, rigorous propagation of uncertainties are a crucial ingredient for any hierarchical inference where the target is the population level distribution.

 

## Fitting the Data -- details

* In order to recover a stellar rotation period from a light curve using a QP-GP, we sample the following posterior PDF

$p({\theta}\,|\,y) \propto \mathcal L({ \theta}) p({ \theta})$

* $y$ are the light-curve ﬂux data
* $\theta$ are the hyperparameters of the kernel
*  $\mathcal L({ \theta})$ is the likelihood function
* $p({ \theta})$ is the prior on the hyperparameters

* Sampling this posterior presents several challenges
 - The likelihood evaluation is computationally expensive.
 - The GP model is very ﬂexible, sometimes at the expense of reliable recovery of the period parameter.
 - The posterior may often be multimodal.

## Likelihood Function

* Simple Gaussian likelihood function, in which uncertainties are Gaussian and uncorrelated

$\ln \mathcal{L} = -\frac{1}{2}\sum_{n=1}^N\left[\frac{(y_n-\mu)^2}{\sigma_n^2}
    + \ln(2\pi\sigma_n^2)\right]$
    
* $y_n$ are the data 
* $\mu$ is the mean model and 
* $\sigma_n$ are the Gaussian uncertainties on the data    
    
    
* Simple Gaussian likelihood function, in which uncertainties are Gaussian and uncorrelated

* The equivalent equation in matrix notation is

$\ln \mathcal{L} = -\frac{1}{2}\bf{r}^T\bf{C}^{-1}\bf{r}-\frac{1}{2}\ln|\bf{C}|
    + \frac{N}{2}\log2\pi $
    
* $r$ is the vector of residuals    
* $C$ is the covariance matrix

 
$\begin{eqnarray}
    \mathbf{C} &=& \left (\begin{array}{cccc}
    \sigma^2_1 & \sigma_{2, 1} & \cdots & \sigma_{N, 1} \\
    \sigma_{1, 2} & \sigma^2_2 & \cdots & \sigma_{N, 2} \\
    && \vdots & \\
    \sigma_{1, N} & \sigma_{2, N} & \cdots & \sigma^2_N
\end{array}\right )
\end{eqnarray}$
 
 
* In the case where the uncertainties are uncorrelated, the noise is ‘white’ (an assumption frequently made by astronomers, sometimes justiﬁed) and the off-diagonal elements of the covariance matrix are zero.

* In the case where there is evidence for correlated ‘noise', as in the case of Kepler light curves, the off-diagonal elements are non-zero.

* With GP regression, a covariance matrix generated by the kernel function $K$ replaces $C$ in the above equation
 -  Specifically, the QP kernel generates K


* Evaluating this likelihood for a large number of points can be computationally expensive.
 - Evaluating L for an entire Kepler light curve (∼40,000 points) takes about ∼5 s using the fast matrix solver HODLR implemented in George
 - Too slow to perform inference on large numbers of light curves
 - Matrix operations necessary to evaluate the GP likelihood scale as $N \ln (N)^2$
  - N is the number of data points

* Accelerate the calculation
 - subsampling the data
 - splitting the light curve into independent sections
 
* Randomly select 1/30th of the points in the full light curve
 - decreases the likelihood evaluation time by a factor of about 50, down to about 100 ms

* Split the light curve into equal-sized chunks containing approximately 300 points per section
 - Evaluate the log-likelihood as the sum of the log-likelihoods of the individual sections
 - All using the same parameters $\theta$
 - Further reduces computation time for a typical light curve (subsampled by a factor of 30) by about a factor of 2, to about 50 ms.
